# DAFF example
This python noteboook shows an example how to use the DAFF class. 
It contains an short introduction to the properties and methods of DAFF objects.
At the end of of this notebook are the records of the object interactively visualized.

## Import of the DAFF class

Before using the DAFF class it must be imported. The file `DAFF.py` contains the class and in this case is located in the same directory as the python notebook.

In [ ]:
from DAFF import DAFF

## Initialization of the DAFF objects

The DAFF class contains all properties of the OpenDAFF. <br>
DAFF objects need the path of the OpenDAFF file as an input parameter. <br>
The needed `.daff` files are located in the following file that can be downloaded from sourceforge: [OpenDAFF-v17](https://sourceforge.net/projects/opendaff/files/OpenDAFF-v17_win32-x64.zip/download). <br>
The paths must be adapted to the location of the files.

In [ ]:
#DAFF objects of two magnitude spectrums of low passes
daff3dB = DAFF('..\\..\\OpenDAFF-v17_win32-x64\\content\\matlab\\ExampleLowPass3dBOmni.ms.daff')
daff6dB = DAFF('..\\..\\OpenDAFF-v17_win32-x64\\content\\matlab\\ExampleLowPass6dBOmni.ms.daff')

#DAFF object from an impulse response 
daffHRIR = DAFF('..\\..\\OpenDAFF-v17_win32-x64\\content\\matlab\\hrtfs\\ITA\\HRTF01_256samples_5x5.v17.ir.daff')

## Accessing the properties of the DAFF objects
The properties of the objects can be easily accessed by the point operator. <br>
Some properties are located in the property `Property` and are defined by the content type of the object. Only spectral content types contain the property `Property.Frequencies`.  <br>
Because the data of metadata can be easily changed, they are contained in a dictionary. <br>
Since the properties are read-only, trying to change them produces an `AttributeError`.

In [ ]:
#Get the content types of the objects
contentType3dB = daff3dB.ContentTypeString
contentTypeHRIR = daffHRIR.ContentTypeString

#Get the samplerate
samplerateHRIR = daffHRIR.Properties.Samplerate

#Print the content types and in the case of the HRIR also its samplerate.
print('The content type of the daff3dB record is ' + contentType3dB + ". ")
print('The records of daffHRIR are ' + contentTypeHRIR + 's and its samplerate is ' + str(samplerateHRIR) + ' Hz.')

#Print the description written in the metadata
print("Description of daff6dB: " + daff3dB.Metadata['DESCRIPTION'])

#Changing the content type or any other property produces an error
try:
    daff3dB.ContentTypeString = 'not possible'
except AttributeError:
    print("An AttributeError occured because this attribute is read-only.")

## Accessing the record data of the DAFF objects

The record of the DAFF object can be obtained by calling the method `GetNearestNeighourRecord`  with the angles of the wanted position as paramters. The nearest neighbour of the position will be searched and its record will be returned.
The record can also directly be obtained by calling the method `GetRecord` with the index of the position as an input parameter. 

In the following example, `daff3dB` and `daff6dB` contain one channel. The records are the magnitude spectrum. `daffHRIR` contains two channels, respectively the impulse response of left and right ear.

In [ ]:
#Angles in degree
azimuth = 34;
elevation = 90;


#Get the record of the nearest neighbour

record6dB, = daff6dB.GetNearestNeighbourRecord(34,65)
recordHRIR = daffHRIR.GetNearestNeighbourRecord(azimuth, elevation)
record3dB, = daff3dB.GetNearestNeighbourRecord(angle2=0,angle1=65)

#daff objects with spectral content types contain the property frequencies
frequencies3dB = daff3dB.Properties.Frequencies
frequencies6dB = daff6dB.Properties.Frequencies

#The number of channels and records
numChannels = daffHRIR.Properties.NumChannels
numRecords = daffHRIR.Properties.NumRecords


print('The object daffHRIR contains ' + str(numRecords) + ' records, each containing ' + str(numChannels) + ' channels.')

print('The magnitude of record3dB at ' + str(frequencies3dB[1]) + ' Hz is ' + str(record3dB[1]) + 
     ' and the magnitude of record6dB at ' + str(frequencies6dB[1]) + ' Hz is ' + str(record6dB[1]) + '. ')
      


---
## Visualization of the data

The data can be easily visualized via plots and the plotted data can be dynamically changed via sliders or other widgets. <br>
### Import libraries
First, we must import `matplotlib.pyplot` for the plots and `ipywidgets` for the widgets. <br> 
The line `%matplotlib notebook` must be added for enabling the live changing of the plots.

In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np

import ipywidgets as widgets

### Initialization of the sliders

The two FloatSliders `sliderAzimuth` & `sliderElevation` set the angles according to their current values. <br>
The attributes of the sliders can be directly passed as initialization parameters.
If the attribute `continous_update` is set to _False_, the widget will only update the value after releasing the pressed mouse button. 

In [ ]:
# Slider for the azimuth angle
sliderAzimuth = widgets.FloatSlider( 
    value = 0,
    description = 'Azimuth:',
    min = -90.0,
    max = 90.0,
    step = 0.5,
    readout = True,
    readout_format = '.1f',
    continuous_update=True,
    layout = widgets.Layout(width='70%', height='80px') )

# Slider for the elevation angle
sliderElevation = widgets.FloatSlider( 
    value = 0,
    description = 'Elevation:',
    min = -90.0,
    max = 90.0,
    step = 0.5,
    readout = True,
    continuous_update=False,
    orientation='horizontal',
    readout_format = '.1f',
    layout = widgets.Layout(width='70%', height='80px'))

### Initialization of the plot lines

The lines containing the impulse responses of the left and right ear are initialized here, before they are plotted in a figure in a later step.

In [ ]:
currentIR = daffHRIR.GetNearestNeighbourRecord(0,0);
samplerate = daffHRIR.Properties.Samplerate;

lineLeftIR = plt.Line2D(np.arange(256)/samplerate,currentIR[0])
lineLeftIR.set_label('left ear IR')
lineLeftIR.set_color("red")

lineRightIR = plt.Line2D(np.arange(256)/samplerate,currentIR[1])
lineRightIR.set_label('right ear IR')

### Definition of the function for changed slider values

The function `OnSliderChanged` sets the y-data of the two lines to the record of the nearest neighour of the current position. <br>
Handled in the function `interactive_output`, `OnSliderChanged` is called after the value of `sliderAzimuth` or `sliderElevation` is changed . 

In [ ]:
#function called on changed slider values 
#get the record of the nearest neighbour of the current angles and plot the ir
def OnSliderChanged(angle1,angle2):
    currentIR = daffHRIR.GetNearestNeighbourRecord(angle1,angle2);
  
    lineLeftIR.set_ydata(currentIR[0])
    lineRightIR.set_ydata(currentIR[1])
    
    
#Interactive output of to sliders that call OnSliderChanged after their values are changed    
output = widgets.interactive_output(OnSliderChanged, {'angle1':sliderAzimuth, 'angle2':sliderElevation});

### Visualization
Here, a figure is plotted and the above initialized lines are added to the figure.

Both slider widgets are in the widget `VBox` and are displayed.

In this example, the azimuth and elevation angle can be changed and the impulse responce of the new nearest neighbour is directly plotted.
The azimuth slider continously updates the plot and the elevation slider fires the event after releasing the pressed mouse button.

In [ ]:
#Plot the figure and add both lines to the axis of the figure.
fig = plt.figure()
fig.suptitle('HRIR example')
ax = fig.add_subplot(111)
ax.add_line(lineLeftIR);
ax.add_line(lineRightIR);
plt.ylabel('Amplitude')
plt.xlabel('Time [s]')

#Set the limits of the axis
ax.set_xlim(0,255/samplerate)
ax.set_ylim(-1.1,1.1)

#Add a legend to the figure
handles, labels = ax.get_legend_handles_labels();
ax.legend(handles, labels);

#Interface
ui = widgets.VBox([sliderAzimuth,sliderElevation]);
display(ui)

# Conversion from time to frequency domain

In this example, the HRIR is converted into the frequency domain.
Thereafter the calculated HRTF is plotted into a new figure.

In [ ]:
currentIR = daffHRIR.GetNearestNeighbourRecord(angle1 = 34,angle2 = 45);
currentTF = [np.fft.fft(currentIR[0]),np.fft.fft(currentIR[1])]
#currentTF = np.fft.fft(record6dB)

numSamples = len(currentTF[0])

f = np.arange(round(numSamples/2))*samplerate/numSamples

In [ ]:
fig = plt.figure()
fig.suptitle('HRTF example')
plt.loglog(f,abs(currentTF[0][:round(numSamples/2)]))
plt.loglog(f,abs(currentTF[1][:round(numSamples/2)]))
plt.ylabel('Magnitude')
plt.xlabel('Frequency [Hz]')
plt.show()
